# Introduction

To create a new trigger in PostgreSQL, you follow these steps:
* First, create a trigger function using the `CREATE FUNCTION` statement.
* Second, bind the trigger function to a table by using the `CREATE TRIGGER` statement.

# Create trigger function syntax

A **trigger function** is similar to a regular user-defined function. However, a trigger function does not take any arguments and has a return value of **type** `TRIGGER`.
```postgresql
CREATE FUNCTION trigger_function()
   RETURNS TRIGGER
   LANGUAGE PLPGSQL
AS $$
BEGIN
   -- trigger logic
END;
$$ 
```

> Note that you can create a trigger function using any language supported by PostgreSQL.

A trigger function receives data about its calling environment through a special structure called `TriggerData`, which contains a set of local variables. For example, `OLD` and `NEW` represent the states of the row in the table before or after the triggering event.

PostgreSQL also provides other local variables preceded by `TG_`, such as `TG_WHEN`, and `TG_TABLE_NAME`.

After creating a trigger function, you can bind it to one or more trigger events such as `INSERT`, `UPDATE`, and `DELETE`.

# CREATE TRIGGER statement

The `CREATE TRIGGER` statement allows you to create a new trigger.

```postgresql
CREATE TRIGGER trigger_name
   {BEFORE | AFTER} { event }
   ON table_name
   [FOR [EACH] { ROW | STATEMENT }]
       EXECUTE PROCEDURE trigger_function 
```
* The **event** can be `INSERT`, `DELETE`, `UPDATE`, or `TRUNCATE`.
* Define the type of triggers, which can be:
    - The **row-level trigger** that is specified by the `FOR EACH ROW` clause.
    - The **statement-level trigger** that is specified by the `FOR EACH STATEMENT` clause.
* Finally, give the name of the trigger function after the `EXECUTE PROCEDURE` keywords.


# CREATE TRIGGER example

**Create a new table called `employees`.**

```postgresql
DROP TABLE IF EXISTS employees;

CREATE TABLE employees(
   id INT GENERATED ALWAYS AS IDENTITY,
   first_name VARCHAR(40) NOT NULL,
   last_name VARCHAR(40) NOT NULL,
   PRIMARY KEY(id)
);
```

**Suppose that when the name of an employee changes, you want to log it in a separate table called `employee_audits`**:

```postgresql
CREATE TABLE employee_audits (
   id INT GENERATED ALWAYS AS IDENTITY,
   employee_id INT NOT NULL,
   last_name VARCHAR(40) NOT NULL,
   changed_on TIMESTAMP NOT NULL
); 
```

**First, create a new function called `log_last_name_changes`**:

```postgresql
CREATE OR REPLACE FUNCTION log_last_name_changes()
  RETURNS TRIGGER
  LANGUAGE PLPGSQL
  AS
$$
BEGIN
	IF NEW.last_name <> OLD.last_name THEN
		 INSERT INTO employee_audits(employee_id,last_name,changed_on)
		 VALUES(OLD.id,OLD.last_name,now());
	END IF;

	RETURN NEW;
END;
$$
```

The function inserts the old last name into the `employee_audits` table, including **employee id**, **last name**, and the **time of change** if the last name of an employee changes.
* The `OLD` represents the row before the update, while the `NEW` represents the new row that will be updated.
* The `OLD.last_name` returns the last name before the update, and the `NEW.last_name` returns the new last name.

**Second, bind the trigger function to the employees table.** 
* The trigger name is `last_name_changes`.
* Before the value of the `last_name` column is updated, the trigger function is automatically invoked to log the changes.

```postgresql
CREATE TRIGGER last_name_changes
  BEFORE UPDATE
  ON employees
  FOR EACH ROW
  EXECUTE PROCEDURE log_last_name_changes(); 
```

**Third, insert some rows into the`employees` table**:

```postgresql
INSERT INTO employees (first_name, last_name)
VALUES ('John', 'Doe');

INSERT INTO employees (first_name, last_name)
VALUES ('Lily', 'Bush');

SELECT * FROM employees; 
```

**Fourth, update `Lily Bush` to `Lily Brown`.**

```postgresql
UPDATE employees
SET last_name = 'Brown'
WHERE ID = 2;

SELECT * FROM employees; 
```

**Finally, verify the contents of the `employee_audits` table**:

```postgresql
SELECT * FROM employee_audits; 
```

The change was logged in the `employee_audits` table by the trigger.